In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

ModuleNotFoundError: ignored

In [ ]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import time

In [ ]:

# Create an 8x8 pixelated image with white background
image = np.ones((8, 8), dtype=np.uint8) * 255

# Set the dots at the corners
image[1, 1] = 0  # Top-left corner
image[1, 6] = 0  # Top-right corner
image[6, 1] = 0  # Bottom-left corner
image[6, 6] = 0  # Bottom-right corner

# Display the image
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.show()



In [ ]:
def to_neqr(image):
    neqr_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            intensity = image[i, j]
            if intensity < 100:
                neqr_image[i, j] = 0
            else:
                neqr_image[i, j] = 255
    return neqr_image

neqr_image = to_neqr(image)

# Display the NEQR state image
plt.imshow(neqr_image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.show()


In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer

def neqr_circuit():
    circuit = QuantumCircuit(2, 2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit

circuit = neqr_circuit()
circuit_drawer(circuit, output='mpl')


In [ ]:
# Display the initial intensity values of the pixel image
print("Initial Intensity Values:")
print(image)





In [ ]:
# Time measurement function
def calculate_execution_time(func):
    start_time = time.time()
    func()
    end_time = time.time()
    execution_time = end_time - start_time
    return execution_time

In [ ]:
# Apply Grover's algorithm to amplify the states of pixels with intensity less than 100
def grover_algorithm(neqr_image):
    num_qubits = int(np.log2(neqr_image.size))  # Number of qubits required based on the NEQR state size
    num_iterations = int(np.sqrt(neqr_image.size))  # Number of iterations for Grover's algorithm


    # Create a quantum circuit with the required number of qubits
    circuit = QuantumCircuit(num_qubits)

    # Initialize the qubits in a superposition of all states
    circuit.h(range(num_qubits))

    # Apply the Oracle to mark the target states
    for i in range(neqr_image.shape[0]):
        for j in range(neqr_image.shape[1]):
            if neqr_image[i, j] == 0:  # Dark shade pixel
                circuit.z(range(num_qubits))

    # Apply the Diffusion operator
    circuit.h(range(num_qubits))
    circuit.z(range(num_qubits))
    circuit.cz(0, num_qubits - 1)
    circuit.h(range(num_qubits))

    # Measure the qubits
    circuit.measure_all()

    return circuit


In [ ]:
# Run Grover's algorithm on the NEQR state
grover_circuit = grover_algorithm(neqr_image)

# Run Grover's algorithm on the NEQR state
execution_time = calculate_execution_time(lambda: grover_algorithm(neqr_image))

# Print the execution time
print("Execution Time: {:.6f} seconds".format(execution_time))

# Simulate the circuit using the Qiskit Aer simulator
simulator = Aer.get_backend('aer_simulator')
job = execute(grover_circuit, simulator, shots=1024)
result = job.result()
counts = result.get_counts(grover_circuit)

# Extract the measurement outcomes as binary strings
outcomes = list(counts.keys())
# Convert the binary strings to decimal values
decimal_outcomes = [int(outcome, 2) for outcome in outcomes]



In [ ]:
# Plot the NEQR state image
plt.imshow(neqr_image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('NEQR State Image')

# Plot the 3D histogram on the pixels
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.arange(neqr_image.shape[1])
y = np.arange(neqr_image.shape[0])
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(neqr_image, dtype=float)

for outcome, count in counts.items():
    decimal_outcome = int(outcome, 2)
    i, j = np.unravel_index(decimal_outcome, neqr_image.shape)
    if neqr_image[i, j] == 0:
        # Display bars only for pixels with intensity less than 100
        Z[i, j] = count
        # Add intensity values as text labels on the bars
        ax.text(j + 0.5, i + 0.5, Z[i, j], str(image[i, j]), color='r')

ax.bar3d(X.ravel(), Y.ravel(), np.zeros_like(X).ravel(), 1, 1, Z.ravel())
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Counts')
ax.set_title('3D Histogram on Pixels')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create an 8x8 pixel image with intensities
image_8x8 = np.array([[30, 80, 120, 200, 150, 90, 70, 10],
                      [40, 100, 90, 180, 140, 100, 50, 20],
                      [20, 70, 130, 220, 180, 120, 60, 30],
                      [50, 90, 140, 190, 160, 110, 80, 40],
                      [60, 100, 110, 170, 130, 80, 40, 10],
                      [70, 120, 100, 160, 120, 70, 30, 20],
                      [80, 130, 90, 140, 100, 60, 20, 50],
                      [90, 140, 80, 130, 90, 50, 10, 60]], dtype=np.uint8)

# Display the original image
plt.imshow(image_8x8, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('Original 8x8 Image')
plt.show()

# Convert the image to NEQR state
def to_neqr(image):
    neqr_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            intensity = image[i, j]
            if intensity < 100:
                neqr_image[i, j] = 0
            else:
                neqr_image[i, j] = 255
    return neqr_image

neqr_image_8x8 = to_neqr(image_8x8)

# Display the NEQR state image
plt.imshow(neqr_image_8x8, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('NEQR State Image (8x8)')
plt.show()


In [ ]:
# Run Grover's algorithm on the NEQR state
grover_circuit_8x8 = grover_algorithm(neqr_image_8x8)

# Run Grover's algorithm on the NEQR state
execution_time_8x8 = calculate_execution_time(lambda: grover_algorithm(neqr_image_8x8))

# Print the execution time
print("Execution Time: {:.6f} seconds".format(execution_time_8x8))

# Simulate the circuit using the Qiskit Aer simulator
simulator = Aer.get_backend('aer_simulator')
job = execute(grover_circuit_8x8, simulator, shots=1024)
result = job.result()
counts = result.get_counts(grover_circuit_8x8)

# Extract the measurement outcomes as binary strings
outcomes = list(counts.keys())
# Convert the binary strings to decimal values
decimal_outcomes = [int(outcome, 2) for outcome in outcomes]



In [ ]:
# Plot the NEQR state image
plt.imshow(neqr_image_8x8, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('NEQR State Image')

# Plot the 3D histogram on the pixels
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.arange(neqr_image_8x8.shape[1])
y = np.arange(neqr_image_8x8.shape[0])
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(neqr_image_8x8, dtype=float)

for outcome, count in counts.items():
    decimal_outcome = int(outcome, 2)
    i, j = np.unravel_index(decimal_outcome, neqr_image_8x8.shape)
    if neqr_image_8x8[i, j] == 0:
        # Display bars only for pixels with intensity less than 100
        Z[i, j] = count
        # Add intensity values as text labels on the bars
        ax.text(j + 0.5, i + 0.5, Z[i, j], str(image_8x8[i, j]), color='r')

ax.bar3d(X.ravel(), Y.ravel(), np.zeros_like(X).ravel(), 1, 1, Z.ravel())
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Counts')
ax.set_title('3D Histogram on Pixels')

plt.show()


In [ ]:
import time
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Convert the image into the NEQR state
def to_neqr(image):
    neqr_image = np.zeros_like(image, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            intensity = image[i, j]
            if intensity < 100:
                neqr_image[i, j] = 0
            else:
                neqr_image[i, j] = 255
    return neqr_image

# Generate a 16x16 pixel image
image = np.random.randint(0, 256, size=(32, 32), dtype=np.uint8)

# Display the initial intensity values of the pixel image
print("Initial Intensity Values:")
print(image)

# Apply Grover's algorithm to amplify the states of pixels with intensity less than 100
def grover_algorithm(neqr_image):
    num_qubits = int(np.log2(neqr_image.size))  # Number of qubits required based on the NEQR state size
    num_iterations = int(np.sqrt(neqr_image.size))  # Number of iterations for Grover's algorithm

    # Create a quantum circuit with the required number of qubits
    circuit = QuantumCircuit(num_qubits)

    # Initialize the qubits in a superposition of all states
    circuit.h(range(num_qubits))

    # Apply the Oracle to mark the target states
    for i in range(neqr_image.shape[0]):
        for j in range(neqr_image.shape[1]):
            if neqr_image[i, j] == 0:  # Dark shade pixel
                circuit.z(range(num_qubits))

    # Apply the Diffusion operator
    circuit.h(range(num_qubits))
    circuit.z(range(num_qubits))
    circuit.cz(0, num_qubits - 1)
    circuit.h(range(num_qubits))

    # Measure the qubits
    circuit.measure_all()

    return circuit

# Convert the image into the NEQR state
neqr_image = to_neqr(image)

# Apply Grover's algorithm on the NEQR state
start_time = time.time()
grover_circuit = grover_algorithm(neqr_image)

# Simulate the circuit using the Qiskit Aer simulator
simulator = Aer.get_backend('aer_simulator')
job = execute(grover_circuit, simulator, shots=1024)
result = job.result()
counts = result.get_counts(grover_circuit)

execution_time = time.time() - start_time

# Extract the measurement outcomes as binary strings
outcomes = list(counts.keys())
# Convert the binary strings to decimal values
decimal_outcomes = [int(outcome, 2) for outcome in outcomes]

# Plot the original image
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('Original Image')

# Plot the NEQR state image
plt.subplot(1, 2, 2)
plt.imshow(neqr_image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('NEQR State Image')

plt.tight_layout()
plt.show()

# Display the execution time
print("Execution Time:", execution_time, "seconds")

# Plot the NEQR state image
plt.imshow(neqr_image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('NEQR State Image')

# Plot the 3D histogram on the pixels
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = np.arange(neqr_image.shape[1])
y = np.arange(neqr_image.shape[0])
X, Y = np.meshgrid(x, y)
Z = np.zeros_like(neqr_image, dtype=float)

for outcome, count in counts.items():
    decimal_outcome = int(outcome, 2)
    i, j = np.unravel_index(decimal_outcome, neqr_image.shape)
    if neqr_image[i, j] == 0:
        # Display bars only for pixels with intensity less than 100
        Z[i, j] = count
        # Add intensity values as text labels on the bars
        ax.text(j + 0.5, i + 0.5, Z[i, j], str(image[i, j]), color='r')

ax.bar3d(X.ravel(), Y.ravel(), np.zeros_like(X).ravel(), 1, 1, Z.ravel())
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Counts')
ax.set_title('3D Histogram on Pixels')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create an 8x8 pixelated image with white background
image = np.ones((8, 8), dtype=np.uint8) * 255

# Set the dots at the corners
image[1, 1] = 0  # Top-left corner
image[1, 6] = 0  # Top-right corner
image[6, 1] = 0  # Bottom-left corner
image[6, 6] = 0  # Bottom-right corner

# Display the image
plt.imshow(image, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.show()
